In [2]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.cloud import storage
from twilio.rest import Client
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
from twilio.rest import Client
import numpy as np
import pandas as pd
import csv
import multiprocessing
import time
'''
def f():
    time.ctime()

def work():
   while 1:
       f()
       time.sleep(10)

if __name__ == '__main__':
    p = multiprocessing.Process(target=work,)
    p.start()
    p.deamon = True
    while 1:
                # Imports the Google Cloud client library

        time.sleep(3)
'''

"\ndef f():\n    time.ctime()\n\ndef work():\n   while 1:\n       f()\n       time.sleep(10)\n\nif __name__ == '__main__':\n    p = multiprocessing.Process(target=work,)\n    p.start()\n    p.deamon = True\n    while 1:\n                # Imports the Google Cloud client library\n\n        time.sleep(3)\n"

In [11]:

def implicit():


    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)
# Instantiates a client
client = language.LanguageServiceClient()
text = u'There is a gun shot in columbia university'
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment
import six
def entities_text(text):
    Event=[]
    Location=[]
    """Detects entities in the text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects entities in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    entities = client.analyze_entities(document).entities

    # entity types from enums.Entity.Type
    entity_type = ('UNKNOWN', 'PERSON', 'LOCATION', 'ORGANIZATION',
                   'EVENT', 'WORK_OF_ART', 'CONSUMER_GOOD', 'OTHER')

    for entity in entities:   
        if entity_type[entity.type]== 'EVENT' or 'OTHER':
            Event.append(entity.name)
        if entity_type[entity.type]== 'LOCATION' or 'ORGANIZATION':
            Location.append(entity.name)


    return Event

account_sid = "ACb46aaaa376515a4c773407c5a4542173"
auth_token = "4d9f0f6dc654fc01b1ca77eb51712347"
client = Client(account_sid, auth_token)
message=[]
for sms in client.messages.list():
    message.append(sms.body)

text0=message[1]
text=[
    'There is a car accident in Trinity School.',
    'There is much noise at Bryant Park.',
    'A car accident happened near Pennsylvania Station.',
    'There is too much noise around the Columbus Circle',
    'A car accident just happened near IFC Center',
    'conflicts happened in NYU Bookstore',
    'conflicts happen near Washington Market Park',
    'car accident near Pace University',
    'car accident near South Ferry',
    'noise in at LGA airport which is near my home',
    'Some people have conflicts in the central park at night which is quite scary',
    'car accident caused the traffic jam near Metropolitan Museum of Art',
    'gun shot near Stuyvesant High School',
    'fire in Central Park'
]
text.append(text0)
Event_and_Location=[]
for x in text:
    Event_and_Location.append(entities_text(x))
Event=[]
Location=[]
for i in Event_and_Location:
    Event.append(i[0])
    Location.append(i[1])
import urllib
import urllib.request
import json
serviceurl = 'http://maps.googleapis.com/maps/api/geocode/json?'
#serviceurl = 'http://python-data.dr-chuck.net/geojson?'
lat=[]
lng=[]
for i in Location:
    while True:
        address=i
        if len(address) < 1:break
        url = serviceurl + urllib.parse.urlencode({'sensor':'false','address':address})
        uh = urllib.request.urlopen(url)
        data = uh.read()
    #接受过来的数据都是字节型数据需要进行decode（）操作
        try: js = json.loads(data.decode())
        except: js = None
    #try:js = json.loads(str(data))
    #except: js = None

        if 'status' not in js or js['status'] != 'OK':
            continue
        lat.append(js['results'][0]['geometry']['location']['lat'])
        lng.append(js['results'][0]['geometry']['location']['lng'])
        location = js['results'][0]['formatted_address']
        break
#    address = input('Enter location:')  

report=[]
for i in range(len(lat)):
    if Event[i]=='fire':
        report.append([Event[i],lat[i],lng[i],8])
    elif Event[i]== 'gun shot':
        report.append([Event[i],lat[i],lng[i],4.5])
    elif Event[i]=='conflicts':
        report.append([Event[i],lat[i],lng[i],3])
    elif Event[i]=='car accident':
        report.append([Event[i],lat[i],lng[i],2.5])
    else:
        report.append([Event[i],lat[i],lng[i],2])
with open('C:/Users/skyof/Desktop/Columbia/Activities/Devfest2018/Devfest21018/test22.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['EVENT','LAT','LNG','R'])
    for i in range(len(lat)):
        row=report[i]
        writer.writerow(row)
